In [3]:
import warnings
warnings.filterwarnings("ignore")

import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from sklearn.model_selection import train_test_split

#import my modules
import acquire as a
import wrangle as w

## Exercises II

As a customer analyst for Telco, you want to know who has spent the most money with the company over their lifetime. You have monthly charges and tenure, so you think you will be able to use those two attributes as features to estimate total charges. You need to do this within an average of $5.00 per customer.

In these exercises, you will complete the first step toward the above goal: acquire and prepare the necessary Telco data from the telco_churn database in the Codeup database server.

**1. Acquire customer_id, monthly_charges, tenure, and total_charges from the telco_churn database for all customers with a 2-year contract.**

In [4]:
#acquire my df using my function but this gives me all the columns 
df = a.get_telco()

In [5]:
#check info
df.head()

,payment_type_id,internet_service_type_id,contract_type_id,customer_id,gender,senior_citizen,partner,dependents,tenure,phone_service,...,tech_support,streaming_tv,streaming_movies,paperless_billing,monthly_charges,total_charges,churn,contract_type,internet_service_type,payment_type
0,2,1,3,0016-QLJIS,Female,0,Yes,Yes,65,Yes,...,Yes,Yes,Yes,Yes,90.45,5957.9,No,Two year,DSL,Mailed check
1,4,1,3,0017-DINOC,Male,0,No,No,54,No,...,Yes,Yes,No,No,45.20,2460.55,No,Two year,DSL,Credit card (automatic)
2,3,1,3,0019-GFNTW,Female,0,No,No,56,No,...,Yes,No,No,No,45.05,2560.1,No,Two year,DSL,Bank transfer (automatic)
3,4,1,3,0056-EPFBG,Male,0,Yes,Yes,20,No,...,Yes,No,No,Yes,39.40,825.4,No,Two year,DSL,Credit card (automatic)
4,3,1,3,0078-XZMHT,Male,0,Yes,No,72,Yes,...,Yes,Yes,Yes,Yes,85.15,6316.2,No,Two year,DSL,Bank transfer (automatic)


In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 7043 entries, 0 to 7042
Data columns (total 24 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   payment_type_id           7043 non-null   int64  
 1   internet_service_type_id  7043 non-null   int64  
 2   contract_type_id          7043 non-null   int64  
 3   customer_id               7043 non-null   object 
 4   gender                    7043 non-null   object 
 5   senior_citizen            7043 non-null   int64  
 6   partner                   7043 non-null   object 
 7   dependents                7043 non-null   object 
 8   tenure                    7043 non-null   int64  
 9   phone_service             7043 non-null   object 
 10  multiple_lines            7043 non-null   object 
 11  online_security           7043 non-null   object 
 12  online_backup             7043 non-null   object 
 13  device_protection         7043 non-null   object 
 14  tech_sup

In [7]:
df.describe().T

,count,mean,std,min,25%,50%,75%,max
payment_type_id,7043.0,2.315633,1.148907,1.00,1.0,2.00,3.00,4.00
internet_service_type_id,7043.0,1.872923,0.737796,1.00,1.0,2.00,2.00,3.00
contract_type_id,7043.0,1.690473,0.833755,1.00,1.0,1.00,2.00,3.00
senior_citizen,7043.0,0.162147,0.368612,0.00,0.0,0.00,0.00,1.00
tenure,7043.0,32.371149,24.559481,0.00,9.0,29.00,55.00,72.00
monthly_charges,7043.0,64.761692,30.090047,18.25,35.5,70.35,89.85,118.75


In [ ]:
#checking the information in this column
df.contract_type_id.value_counts()

In [ ]:
df.contract_type.value_counts()

In [ ]:
#getting only the customers who have 2 year contract using the condition df.contract_type_id == 3
telco_df = df[['customer_id', 'monthly_charges', 'tenure', 'total_charges']][df.contract_type_id == 3]
telco_df.head()

In [ ]:
telco_df.shape

____________

In [8]:
#other way to do it is if I have a generic function so I can use a different query
query = """
        SELECT 
            customer_id, 
            monthly_charges, 
            tenure, 
            total_charges
        FROM customers
        WHERE contract_type_id = 3;
        """

df2 = a.get_data_from_sql('telco_churn', query)

In [9]:
df2.head(1)

,customer_id,monthly_charges,tenure,total_charges
0,0013-SMEOE,109.7,71,7904.25


In [10]:
df2.shape

(1695, 4)

In [11]:
df2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1695 entries, 0 to 1694
Data columns (total 4 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   customer_id      1695 non-null   object 
 1   monthly_charges  1695 non-null   float64
 2   tenure           1695 non-null   int64  
 3   total_charges    1695 non-null   object 
dtypes: float64(1), int64(1), object(2)
memory usage: 53.1+ KB


In [ ]:
df2.describe()

___________________

**Takeaways**
- customer_id and otal_charges are object data type. for total_charges should be float type. 
- tenure has a minimum value of 0. that means new customers
- I don't seem to have any Null values

_____________________

**2. Using your acquired Telco data, walk through the summarization and cleaning steps in your wrangle.ipynb file like we did above. You may handle the missing values however you feel is appropriate and meaningful; remember to document your process and decisions using markdown and code commenting where helpful.**

In [ ]:
#checking information of the columns. we noticed  total_charges is object type
telco_df.info()

In [ ]:
#drop duplicates
telco_df = telco_df.drop_duplicates()

In [ ]:
# Find the total number of Null values in each column of our DataFrame.
telco_df.isnull().sum()

In [ ]:
# Check for any Null values in each column of our DataFrame.

telco_df.isnull().any()

In [ ]:
# Return the names for any columns in our DataFrame with any Null values.

df.columns[df.isnull().any()]

In [ ]:
#trying to change total_charges to float
#telco_df['total_charges'].astype('float')
#when we run the code to convert to float, we get an error (could not convert string to float: ''), there is a space 
#in some values

In [ ]:
#these are the observations that have space in total charges
#these customers are new customers becuase they have a tenure of 0 and have not done their first payment. 
telco_df[telco_df['total_charges']== ' ']

In [ ]:

# I wll add a '0' only to these customers
#
telco_df[telco_df['total_charges']== ' '] = telco_df[telco_df['total_charges']== ' '].replace(' ','0')

In [ ]:
#checking the info
telco_df[telco_df['total_charges']== '0']

In [ ]:
#now I can convert total_charges to float
telco_df['total_charges']= telco_df['total_charges'].astype('float')

In [ ]:
telco_df.info()

In [ ]:
telco_df.shape

____________

What if the total charges that are 0 I  change them to 1 since they were probably customers for about a month.

In [ ]:
df2.tenure.value_counts().sort_index().head()

In [ ]:
# Replace any tenures of 0 with 1.

df2.tenure = df2.tenure.replace(0, 1)

In [ ]:

# Validate my tenure count for value 1; I went from two to 12.

df2.tenure.value_counts().sort_index().head()

In [ ]:
# These observations also need total_charges handled. 
# The other tenure 1 observations have same monthly and total charges.

df2[df2.tenure == 1]

In [ ]:
# Replace the blank total_charges with the monthly_charge for tenure == 1.

df.total_charges = np.where(df.total_charges==' ', df.monthly_charges, df.total_charges)

In [ ]:
# Validate my changes.

df[df.tenure == 1]

**3. End with a wrangle.py file that contains the necessary functions to automate your process from acquiring the data to returning a cleaned dataframe with no missing values. Name your final function wrangle_telco.**

In [ ]:

# Handle the data type for total_charges.

df.total_charges = df.total_charges.astype(float)

In [ ]:
# Validate my data type conversion.

df.info()

____________

In [ ]:
#creating my functions

In [ ]:
def clean_telco(df):
    ''''
    This function will get customer_id, monthly_charges, tenure, and total_charges 
    from the previously acquired telco df, for all customers with a 2-year contract.
    drop any duplicate observations, 
    conver total_charges to a float type.
    return cleaned telco DataFrame
    '''
    #getting only the customers who have 2 year contract using the condition df.contract_type_id == 3
    telco_df = df[['customer_id', 'monthly_charges', 'tenure', 'total_charges']][df.contract_type_id == 3]
    #drop duplicates
    telco_df = telco_df.drop_duplicates()
    # add a '0' only to the columns that have " "
    telco_df[telco_df['total_charges']== ' '] = telco_df[telco_df['total_charges']== ' '].replace(' ','0')
    # convert total_charges to float
    telco_df['total_charges']= telco_df['total_charges'].astype('float')
        
    return telco_df

In [ ]:
def split_data(df):
    '''
    take in a DataFrame and return train, validate, and test DataFrames.
    
    '''
    train_validate, test = train_test_split(df, test_size=.2, random_state=123)
    train, validate = train_test_split(train_validate, 
                                       test_size=.3, 
                                       random_state=123)
    print(f'train -> {train.shape}')
    print(f'validate -> {validate.shape}')
    print(f'test -> {test.shape}')
    return train, validate, test

In [ ]:
def wrangle_telco():
    ''''
    This function will acquire telco db using get_telco function. then it will use another
    function named  clean_telco that create a new df only with  customer_id, monthly_charges, tenure, and total_charges 
    from the previously acquired telco df, this new df will contain only customers with a 2-year contract.
    drop any duplicate observations, 
    conver total_charges to a float type.
    return cleaned telco DataFrame
    '''
    df = acquire.get_telco()
    telco_df = clean_telco(df)
    return telco_df
    

In [ ]:
#using my functions that are in wrangle.py

In [2]:
df = w.wrangle_telco()

In [3]:
df.head()

,customer_id,monthly_charges,tenure,total_charges
0,0016-QLJIS,90.45,65,5957.90
1,0017-DINOC,45.20,54,2460.55
2,0019-GFNTW,45.05,56,2560.10
3,0056-EPFBG,39.40,20,825.40
4,0078-XZMHT,85.15,72,6316.20


In [4]:
df.shape

(1695, 4)

In [5]:
train, validate, test = w.split_data(df)

train -> (949, 4)
validate -> (407, 4)
test -> (339, 4)


## Exercises III - Challenge

Let's set up an example scenario as perspective for our regression exercises using the Zillow dataset.

As a Codeup data science graduate, you want to show off your skills to the Zillow data science team in hopes of getting an interview for a position you saw pop up on LinkedIn. You thought it might look impressive to build an end-to-end project in which you use some of their Kaggle data to predict property values using some of their available features; who knows, you might even do some feature engineering to blow them away. Your goal is to predict the values of single unit properties using the obervations from 2017.

In these exercises, you will complete the first step toward the above goal: acquire and prepare the necessary Zillow data from the zillow database in the Codeup database server.

**1. Acquire bedroomcnt, bathroomcnt, calculatedfinishedsquarefeet, taxvaluedollarcnt, yearbuilt, taxamount, and fips from the zillow database for all 'Single Family Residential' properties.**

In [12]:
zillow_df= a.get_new_zillow()

In [13]:
#I'm doing this because it takes a long time to get the db from sql
z_df = zillow_df

In [14]:
zillow_df.head()

,bedroomcnt,bathroomcnt,calculatedfinishedsquarefeet,taxvaluedollarcnt,yearbuilt,taxamount,fips
0,0.0,0.0,NaN,27516.0,NaN,NaN,6037.0
1,0.0,0.0,NaN,10.0,NaN,NaN,6037.0
2,0.0,0.0,NaN,10.0,NaN,NaN,6037.0
3,0.0,0.0,NaN,2108.0,NaN,174.21,6037.0
4,4.0,2.0,3633.0,296425.0,2005.0,6941.39,6037.0


In [15]:

zillow_df.shape

(2152863, 7)

In [16]:
zillow_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2152863 entries, 0 to 2152862
Data columns (total 7 columns):
 #   Column                        Dtype  
---  ------                        -----  
 0   bedroomcnt                    float64
 1   bathroomcnt                   float64
 2   calculatedfinishedsquarefeet  float64
 3   taxvaluedollarcnt             float64
 4   yearbuilt                     float64
 5   taxamount                     float64
 6   fips                          float64
dtypes: float64(7)
memory usage: 115.0 MB


In [ ]:
zillow_df.info(null_counts=True)

In [ ]:
df.describe().T

2. Using your acquired Zillow data, walk through the summarization and cleaning steps in your wrangle.ipynb file like we did above. You may handle the missing values however you feel is appropriate and meaninful; remember to document your process and decisions using markdown and code commenting where helpful.

In [ ]:
# checking the info 
zillow_df.info()

In [ ]:
# this shows us non-nulls
zillow_df.info(null_counts=True)

In [68]:

def miss_dup_values(df):
    '''
    this function takes a dataframe as input and will output metrics for missing values and duplicated rows, 
    and the percent of that column that has missing values and duplicated rows
    '''
        # Total missing values
    mis_val = df.isnull().sum()
        # Percentage of missing values
    mis_val_percent = 100 * df.isnull().sum() / len(df)
        #total of duplicated
    dup = df.duplicated().sum()  
        # Percentage of missing values
    dup_percent = 100 * dup / len(df)
        # Make a table with the results
    mis_val_table = pd.concat([mis_val, mis_val_percent], axis=1)
        # Rename the columns
    mis_val_table_ren_columns = mis_val_table.rename(columns = {0 : 'Missing Values', 1 : '% of Total Values'})
        # Sort the table by percentage of missing descending
    mis_val_table_ren_columns = mis_val_table_ren_columns[
    mis_val_table_ren_columns.iloc[:,1] != 0].sort_values('% of Total Values', ascending=False).round(1)
        # Print some summary information
    print ("Your selected dataframe has " + str(df.shape[1]) + " columns.\n"      
           "There are " + str(mis_val_table_ren_columns.shape[0]) +
           " columns that have missing values.")
    print( "  ")
    print (f"** There are {dup} duplicate rows that represents {round(dup_percent, 2)}% of total Values**")
        # Return the dataframe with missing information
    return mis_val_table_ren_columns

In [69]:
#using the function above
miss_dup_values(zillow_df)

Your selected dataframe has 7 columns.
There are 0 columns that have missing values.
  
** There are 0 duplicate rows that represents 0.0% of total Values**


,Missing Values,% of Total Values


In [24]:
# this shows us non-nulls
zillow_df.info(null_counts=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2152863 entries, 0 to 2152862
Data columns (total 7 columns):
 #   Column                        Non-Null Count    Dtype  
---  ------                        --------------    -----  
 0   bedroomcnt                    2152852 non-null  float64
 1   bathroomcnt                   2152852 non-null  float64
 2   calculatedfinishedsquarefeet  2144379 non-null  float64
 3   taxvaluedollarcnt             2152370 non-null  float64
 4   yearbuilt                     2143526 non-null  float64
 5   taxamount                     2148421 non-null  float64
 6   fips                          2152863 non-null  float64
dtypes: float64(7)
memory usage: 115.0 MB


In [70]:
zillow_df['bedroomcnt'].nunique()

19

In [36]:
zillow_df['bedroomcnt'].value_counts().sort_index()

0.0      13187
1.0      23166
2.0     335473
3.0     964298
4.0     634289
5.0     150866
6.0      25166
7.0       4807
8.0       1107
9.0        291
10.0       121
11.0        34
12.0        12
13.0        16
14.0         7
15.0         6
16.0         2
18.0         3
25.0         1
Name: bedroomcnt, dtype: int64

In [37]:
zillow_df[zillow_df['bedroomcnt']== 0].head()

,bedroomcnt,bathroomcnt,calculatedfinishedsquarefeet,taxvaluedollarcnt,yearbuilt,taxamount,fips
0,0.0,0.0,NaN,27516.0,NaN,NaN,6037.0
1,0.0,0.0,NaN,10.0,NaN,NaN,6037.0
2,0.0,0.0,NaN,10.0,NaN,NaN,6037.0
3,0.0,0.0,NaN,2108.0,NaN,174.21,6037.0
5,0.0,0.0,NaN,124.0,NaN,NaN,6037.0


In [38]:
zillow_df.shape

(2152863, 7)

In [39]:
# drop duplicates
zillow_df = zillow_df.drop_duplicates()
zillow_df.shape

(2141219, 7)

In [48]:
#drop nulls
zillow_df = zillow_df.dropna(how='any',axis=0)

In [52]:
#size after the drops
zillow_df.shape

(2130214, 7)

In [49]:
missing_values_table(zillow_df)

Your selected dataframe has 7 columns.
There are 0 columns that have missing values.
** There are 0 duplicate rows that represents 0.0% of total Values**


,Missing Values,% of Total Values


In [51]:
#confirm that we dond have nulls
zillow_df.info(null_counts=True)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2130214 entries, 4 to 2152862
Data columns (total 7 columns):
 #   Column                        Non-Null Count    Dtype  
---  ------                        --------------    -----  
 0   bedroomcnt                    2130214 non-null  float64
 1   bathroomcnt                   2130214 non-null  float64
 2   calculatedfinishedsquarefeet  2130214 non-null  float64
 3   taxvaluedollarcnt             2130214 non-null  float64
 4   yearbuilt                     2130214 non-null  float64
 5   taxamount                     2130214 non-null  float64
 6   fips                          2130214 non-null  float64
dtypes: float64(7)
memory usage: 130.0 MB


In [54]:
zillow_df.describe().T

,count,mean,std,min,25%,50%,75%,max
bedroomcnt,2130214.0,3.301635,0.932826,0.00,3.00,3.00,4.00,25.00
bathroomcnt,2130214.0,2.240947,0.991584,0.00,2.00,2.00,3.00,32.00
calculatedfinishedsquarefeet,2130214.0,1864.457894,1223.700291,1.00,1258.00,1624.00,2210.00,952576.00
taxvaluedollarcnt,2130214.0,461250.206241,678402.452862,22.00,189625.25,328425.50,534656.75,90188462.00
yearbuilt,2130214.0,1960.912632,22.148424,1801.00,1949.00,1958.00,1975.00,2016.00
taxamount,2130214.0,5624.179572,7828.718383,6.34,2546.94,4112.86,6414.56,1078101.87
fips,2130214.0,6048.242231,20.297915,6037.00,6037.00,6037.00,6059.00,6111.00


**takeaways**
there are houses with ) bedrooms, bathrooms and calculatedfinishedsquarefeet = 1 ??


In [85]:
#let see value counts in each column
cols = zillow_df.columns.to_list()
for col in cols:
    print(col)
    print(zillow_df[col].value_counts().sort_index().head(26))

bedroomcnt
0.0       4259
1.0      22713
2.0     333208
3.0     958078
4.0     630289
5.0     150194
6.0      25092
7.0       4791
8.0       1103
9.0        290
10.0       118
11.0        34
12.0        12
13.0        15
14.0         7
15.0         5
16.0         2
18.0         3
25.0         1
Name: bedroomcnt, dtype: int64
bathroomcnt
0.00       4199
0.50         16
1.00     411264
1.50      30275
1.75          3
2.00     938213
2.50     141089
3.00     421066
3.50      28286
4.00      81903
4.50      19396
5.00      28299
5.50       6174
6.00      10717
6.50       1330
7.00       4381
7.50        382
8.00       1681
8.50        108
9.00        707
9.50         50
10.00       322
10.50        14
11.00       145
11.50         3
12.00        73
Name: bathroomcnt, dtype: int64
calculatedfinishedsquarefeet
1.0      6
2.0      2
3.0      7
7.0      1
10.0     2
12.0     1
20.0    15
30.0     3
31.0     1
32.0     1
40.0     9
43.0     1
48.0     2
54.0     1
56.0     1
60.0     8
63.0    

In [89]:
#bedrooms, yearbuilt and fips can be converted to int
zillow_df[['bedroomcnt', 'yearbuilt', 'fips']] = zillow_df[['bedroomcnt', 'yearbuilt', 'fips']].astype(int)

In [90]:
zillow_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2130214 entries, 4 to 2152862
Data columns (total 7 columns):
 #   Column                        Dtype  
---  ------                        -----  
 0   bedroomcnt                    int64  
 1   bathroomcnt                   float64
 2   calculatedfinishedsquarefeet  float64
 3   taxvaluedollarcnt             float64
 4   yearbuilt                     int64  
 5   taxamount                     float64
 6   fips                          int64  
dtypes: float64(4), int64(3)
memory usage: 130.0 MB


In [91]:
zillow_df[zillow_df.calculatedfinishedsquarefeet < 10]

,bedroomcnt,bathroomcnt,calculatedfinishedsquarefeet,taxvaluedollarcnt,yearbuilt,taxamount,fips
58438,2,1.0,1.0,121376.0,1907,1996.35,6037
122963,2,2.0,3.0,584804.0,1955,6822.61,6037
402516,1,4.0,3.0,356717.0,1986,3840.63,6037
584383,5,2.0,3.0,139441.0,1958,1676.86,6037
587783,4,3.0,2.0,1219502.0,1952,14299.74,6037
683031,2,3.0,3.0,981300.0,1950,10657.81,6037
724785,5,7.0,7.0,1653839.0,1925,20140.20,6037
821592,1,4.0,3.0,1827450.0,1991,22011.62,6037
1046787,0,0.0,1.0,28091.0,1963,439.55,6037
1276353,1,3.0,1.0,124906.0,1953,2020.66,6037


In [ ]:
#this makes no sense. a house with more that one bedroom and has less than 10 squarefeet

In [97]:
zillow_df.nunique()

bedroomcnt                          19
bathroomcnt                         38
calculatedfinishedsquarefeet     10557
taxvaluedollarcnt               588178
yearbuilt                          153
taxamount                       915868
fips                                 3
dtype: int64

In [104]:
zillow_df[zillow_df.calculatedfinishedsquarefeet < 500].count()

bedroomcnt                      4736
bathroomcnt                     4736
calculatedfinishedsquarefeet    4736
taxvaluedollarcnt               4736
yearbuilt                       4736
taxamount                       4736
fips                            4736
dtype: int64

In [107]:
zillow_df[(zillow_df.bedroomcnt == 0) & (zillow_df.bathroomcnt == 0)].sort_values('calculatedfinishedsquarefeet').head(50)

,bedroomcnt,bathroomcnt,calculatedfinishedsquarefeet,taxvaluedollarcnt,yearbuilt,taxamount,fips
1046787,0,0.0,1.0,28091.0,1963,439.55,6037
639470,0,0.0,10.0,8033.0,1984,221.25,6037
1098133,0,0.0,20.0,9043.0,2005,156.43,6037
2088114,0,0.0,20.0,7246.0,2005,135.02,6037
193749,0,0.0,20.0,3388.0,2005,89.03,6037
1894529,0,0.0,20.0,3045.0,2005,84.95,6037
53614,0,0.0,20.0,1124.0,2005,62.05,6037
1937466,0,0.0,20.0,1687.0,2005,68.76,6037
118391,0,0.0,20.0,3454.0,2005,89.82,6037
1044244,0,0.0,20.0,1105.0,2005,61.82,6037


**takeaways**
- dropped duplicated rows
- droped rows that have nulls
- there are houses with 0 bedrooms, bathrooms and calculatedfinishedsquarefeet = 1,2, ??



3. Store all of the necessary functions to automate your process from acquiring the data to returning a cleaned dataframe witn no missing values in your wrangle.py file. Name your final function wrangle_zillow.